In [4]:
%pip install numpy scikit-learn

  Using cached numpy-2.2.3-cp313-cp313-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached scikit_learn-1.6.1-cp313-cp313-macosx_10_13_x86_64.whl.metadata (31 kB)
  Using cached scipy-1.15.2-cp313-cp313-macosx_14_0_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached numpy-2.2.3-cp313-cp313-macosx_14_0_x86_64.whl (6.7 MB)
Using cached scikit_learn-1.6.1-cp313-cp313-macosx_10_13_x86_64.whl (12.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp313-cp313-macosx_14_0_x86_64.whl (25.2 MB)
Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [6]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [7]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [8]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [9]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [10]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [11]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [12]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [13]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [14]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [26]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [16]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [17]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ], shape=(11314,))

In [18]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  6385,  1149, 11238], shape=(11314,))

In [19]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [20]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [21]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [22]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [23]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [24]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506


---
# Resolución
_Autor: Prof. Alejandro Lloveras_

### Consigna del desafío 1

**1.** Vectorizar documentos.
  1. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
  2. Estudiar los 5 documentos más similares de cada uno
  3. Analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

**2.** Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test.
  - Considerar cambiar parámteros de instanciación del vectorizador y los modelos
  - Probar modelos de Naïve Bayes Multinomial y ComplementNB.

**3.** Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras.
  - Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.
  - **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [27]:
import random

## 1. Vectorizar documentos

### 1.1 Selección al azar y medida de similaridad

Selección de 5 documentos al azar

In [49]:
documents = []
for i in range(5):
    number = random.randint(1,len(newsgroups_train.data))
    documents.append(number)
print(documents)

[4336, 3933, 4273, 9733, 3375]


Previsualización del contenido para cada documento:

In [82]:
for idx in documents:
    clase = newsgroups_train.target_names[y_train[idx]] # Consultamos la clase para cada documento
    print("• Indice:", idx, ", clase:", clase, end="\n\n")
    print(newsgroups_train.data[idx][:500],'(…)',end="\n\n\n")

• Indice: 4336 , clase: sci.electronics

I have a MOSFET pulled out of a Trygon power supply, for which I have no 
manual.  It's a Motorola part with a 1972 date code and the number

		285-4 (…)


• Indice: 3933 , clase: rec.sport.baseball

I was wondering if anyone had any kind of Fenway Park gif.
I would appreciate it if someone could send me one.
Thanks in advance. (…)


• Indice: 4273 , clase: comp.sys.ibm.pc.hardware

Hi all,  
   
    
    Could anybody please tell me where I might be able to find device drivers 
for a couple of older Gateway ethernet cards?  I don't have the model number
off hand, but they have only a BNC connector, and a header connector for 
a Novell keycard (one has one installed).
 
    I'm looking at using these with a 2 node copy of 10-net that I picked up
at a swap meet.  (I'd love to do Lantastic or Netware lite, but I'm a poor
college student and the price was right.)

    Please r (…)


• Indice: 9733 , clase: comp.windows.x

Hi,

I am about to write a

### 1.2 Similaridad de documentos

In [ ]:
for idx in documents:
    clase = newsgroups_train.target_names[y_train[idx]] # Consultamos la clase para cada documento
    print("Indice:", idx, ", clase:", clase)
    cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
    simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
    mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
    print("Documentos similares:")
    for doc, i in enumerate(mostsim):
        print(f"  - {i} ({simil[doc]:.4f}),", newsgroups_train.target_names[y_train[i]])
    print()

Indice: 4336 , clase: sci.electronics
Documentos similares:
  - 5067 (0.1807), sci.electronics
  - 8943 (0.1687), misc.forsale
  - 1033 (0.1684), misc.forsale
  - 314 (0.1577), comp.sys.ibm.pc.hardware
  - 7445 (0.1533), sci.electronics

Indice: 3933 , clase: rec.sport.baseball
Documentos similares:
  - 1967 (0.3019), rec.sport.baseball
  - 9858 (0.2941), comp.os.ms-windows.misc
  - 489 (0.2240), comp.sys.mac.hardware
  - 10756 (0.2209), soc.religion.christian
  - 5515 (0.2191), comp.graphics

Indice: 4273 , clase: comp.sys.ibm.pc.hardware
Documentos similares:
  - 7812 (0.1868), comp.os.ms-windows.misc
  - 3166 (0.1852), soc.religion.christian
  - 9623 (0.1828), talk.politics.mideast
  - 1292 (0.1785), talk.politics.mideast
  - 5375 (0.1779), comp.os.ms-windows.misc

Indice: 9733 , clase: comp.windows.x
Documentos similares:
  - 367 (0.4727), comp.windows.x
  - 2676 (0.3910), comp.windows.x
  - 2467 (0.3910), comp.windows.x
  - 1274 (0.3836), comp.windows.x
  - 6657 (0.3628), comp.win

Podemos apreciar que los documentos similares pertenecen a clases relacionadas.

### 1.3 Análisis de similaridad

#### 1. Documento 4336 (`sci.electronics`):

In [ ]:
sel = 0 # modificar para estudiar cada documento de ejemplo (rango 0-4)
idx = documents[sel]

print("DOCUMENTO ANALIZADO:", idx)
clase = newsgroups_train.target_names[y_train[idx]]
print("Clase:", clase)
print("Contenido:")
print(f'"{newsgroups_train.data[idx]}"',end="\n\n")

# Repetimos el proceso anterior
cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
print("SIMILARES:")
for doc, i in enumerate(mostsim):
    print(f"#{doc} Índice {i} | Similaridad: {simil[doc]:.4f}")
    print("Clase:", newsgroups_train.target_names[y_train[i]])
    print("Contenido:")
    print(f'"{newsgroups_train.data[i]}"',end="\n\n")

DOCUMENTO ANALIZADO: 4336
Clase: sci.electronics
Contenido:
"I have a MOSFET pulled out of a Trygon power supply, for which I have no 
manual.  It's a Motorola part with a 1972 date code and the number

		285-4"

SIMILARES:
#0 Índice 5067 | Similaridad: 0.1807
Clase: sci.electronics
Contenido:
"


Since your MOSFET is a 1972 vintage, it's probably not a very good one by 
today's standards.  If you have an idea about its voltage and current 
ratings, e.g. 60VDC @ 6A, you can probably get away with replacing it with 
anything with better specs.  Early MOSFETS had a gate-source voltage rating 
of approximately +/- 20 VDCmax, and they would usually turn completely "ON" 
at +10VDC.  Otherwise, MOSFETS are not really mysterious -- they're more or 
less voltage controlled current sources.  If the MOSFET in your circuit is 
used as an open-loop, voltage controlled current source, you may have to 
experiment with various gain-altering techniques."

#1 Índice 8943 | Similaridad: 0.1687
Clase: mi

**Análisis:**
- **Original:** habla sobre componentes electrónicos, transistores MOSFET de una fuente de alimentación.
    - _Términos clave: MOSFET, 1972, Motorola, power supply, manual._
- **Similitud 0** (`sci.electronics`): descripción sobre tecnología MOSFET.
    - _Términos en común: MOSFET, 1972, Motorola_
- **Similitud 1** (`misc.forsale`): anuncio publicitario de componentes electrónicos para radio.
    - _Términos en común: Motorola, power supply, manual._
- **Similitud 2** (`misc.forsale`): anuncio publicitario de transmisor de radio Motorola.
    - _Términos en común: power supply, manual._
- **Similitud 3** (`comp.sys.ibm.pc.hardware`): ofrece repuestos para equipos IBM.
    - _Términos en común: power supply._
- **Similitud 4** (`sci.electronics`): describe un problema técnico con un modem.
    - _Términos en común: power supply._

#### 2. Documento 3933 (`rec.sport.baseball`):

In [113]:
sel = 1 # modificar para estudiar cada documento de ejemplo (rango 0-4)
idx = documents[sel]

print("DOCUMENTO ANALIZADO:", idx)
clase = newsgroups_train.target_names[y_train[idx]]
print("Clase:", clase)
print("Contenido:")
print(f'"{newsgroups_train.data[idx]}"',end="\n\n")

# Repetimos el proceso anterior
cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
print("SIMILARES:")
for doc, i in enumerate(mostsim):
    print(f"#{doc} Índice {i} | Similaridad: {simil[doc]:.4f}")
    print("Clase:", newsgroups_train.target_names[y_train[i]])
    print("Contenido:")
    print(f'"{newsgroups_train.data[i]}"',end="\n\n")

DOCUMENTO ANALIZADO: 3933
Clase: rec.sport.baseball
Contenido:
"I was wondering if anyone had any kind of Fenway Park gif.
I would appreciate it if someone could send me one.
Thanks in advance."

SIMILARES:
#0 Índice 1967 | Similaridad: 0.3019
Clase: rec.sport.baseball
Contenido:
"
Me too!  I would like any park or action gif or jpeg about baseball."

#1 Índice 9858 | Similaridad: 0.2941
Clase: comp.os.ms-windows.misc
Contenido:
"Hi! I was wondering if anyone out there could help me.
I have an error message that goes:



What does it mean?

I am running MS windows 3.1.

Thanks in advance"

#2 Índice 489 | Similaridad: 0.2240
Clase: comp.sys.mac.hardware
Contenido:
"A few posts back, somebody mentioned that the Duo might crash if it has
the wrong kind (non-self refreshing) of RAM in it.  My Duo crashes
sometimes after sleep, and I am wondering if there is any software which
will tell me whether or not I have the right kind of RAM installed.  I
had thought that the problem was the batter


**Análisis:**
- **Original:** 
    - _Términos clave: _
- **Similitud 0** (`a`): 
    - _Términos en común: _
- **Similitud 1** (`a`): 
    - _Términos en común: _
- **Similitud 2** (`a`): 
    - _Términos en común: _
- **Similitud 3** (`a`): 
    - _Términos en común: _
- **Similitud 4** (`a`): 
    - _Términos en común: _

#### 3. Documento 4273 (`comp.sys.ibm.pc.hardware`):

In [116]:
sel = 2 # modificar para estudiar cada documento de ejemplo (rango 0-4)
idx = documents[sel]

print("DOCUMENTO ANALIZADO:", idx)
clase = newsgroups_train.target_names[y_train[idx]]
print("Clase:", clase)
print("Contenido:")
print(f'"{newsgroups_train.data[idx]}"',end="\n\n")

# Repetimos el proceso anterior
cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
print("SIMILARES:")
for doc, i in enumerate(mostsim):
    print(f"#{doc} Índice {i} | Similaridad: {simil[doc]:.4f}")
    print("Clase:", newsgroups_train.target_names[y_train[i]])
    print("Contenido:")
    print(f'"{newsgroups_train.data[i]}"',end="\n\n")

DOCUMENTO ANALIZADO: 4273
Clase: comp.sys.ibm.pc.hardware
Contenido:
"Hi all,  
   
    
    Could anybody please tell me where I might be able to find device drivers 
for a couple of older Gateway ethernet cards?  I don't have the model number
off hand, but they have only a BNC connector, and a header connector for 
a Novell keycard (one has one installed).
 
    I'm looking at using these with a 2 node copy of 10-net that I picked up
at a swap meet.  (I'd love to do Lantastic or Netware lite, but I'm a poor
college student and the price was right.)

    Please reply via email, as I haven't had a lot of time for news because
of exams and such.  "

SIMILARES:
#0 Índice 7812 | Similaridad: 0.1868
Clase: comp.os.ms-windows.misc
Contenido:
"
I have used both version 1.17 drivers for Win 3.1 and the new 2.03 drivers.
I have had none of these problems.  No GPF's at all.  I have a feeling that
your problems are not with the card or drivers.  The ATI Ultra drivers are
considered some of the m


**Análisis:**
- **Original:** 
    - _Términos clave: _
- **Similitud 0** (`a`): 
    - _Términos en común: _
- **Similitud 1** (`a`): 
    - _Términos en común: _
- **Similitud 2** (`a`): 
    - _Términos en común: _
- **Similitud 3** (`a`): 
    - _Términos en común: _
- **Similitud 4** (`a`): 
    - _Términos en común: _

#### 4. Documento 9733 (`comp.windows.x`):

In [115]:
sel = 3 # modificar para estudiar cada documento de ejemplo (rango 0-4)
idx = documents[sel]

print("DOCUMENTO ANALIZADO:", idx)
clase = newsgroups_train.target_names[y_train[idx]]
print("Clase:", clase)
print("Contenido:")
print(f'"{newsgroups_train.data[idx]}"',end="\n\n")

# Repetimos el proceso anterior
cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
print("SIMILARES:")
for doc, i in enumerate(mostsim):
    print(f"#{doc} Índice {i} | Similaridad: {simil[doc]:.4f}")
    print("Clase:", newsgroups_train.target_names[y_train[i]])
    print("Contenido:")
    print(f'"{newsgroups_train.data[i]}"',end="\n\n")

DOCUMENTO ANALIZADO: 9733
Clase: comp.windows.x
Contenido:
"Hi,

I am about to write an application in X/Motif that will require the
embedding of a pseudo tty. So, before I re-invent the wheel, has anyone
written/gotten a motif widget that does the job ? Otherwise, I would
appreciate any pointers to make such a beast.

My environment is X11R4/Motif 1.1 and X11R5/Motif 1.2 (if this helps).


Thanks in advance.
"

SIMILARES:
#0 Índice 367 | Similaridad: 0.4727
Clase: comp.windows.x
Contenido:
"
  Let me add another of my concerns: Yes, I can buy a port of Motif for "cheap",
but I cannot get the source for "cheap", hence I am limited to using whatever X
libraries the Motif port was compiled against (at least with older versions of
Motif. I have been told that Motif 1.2 can be used with any X, but I have not
seen it myself).

  Currently, I have X11R5 running on eight different unix platforms, of which
only three came with Motif. On those three, I am unable to use the X11R5
libraries to bu


**Análisis:**
- **Original:** 
    - _Términos clave: _
- **Similitud 0** (`a`): 
    - _Términos en común: _
- **Similitud 1** (`a`): 
    - _Términos en común: _
- **Similitud 2** (`a`): 
    - _Términos en común: _
- **Similitud 3** (`a`): 
    - _Términos en común: _
- **Similitud 4** (`a`): 
    - _Términos en común: _

#### 5. Documento 3375 (`comp.sys.ibm.pc.hardware`):

In [114]:
sel = 4 # modificar para estudiar cada documento de ejemplo (rango 0-4)
idx = documents[sel]

print("DOCUMENTO ANALIZADO:", idx)
clase = newsgroups_train.target_names[y_train[idx]]
print("Clase:", clase)
print("Contenido:")
print(f'"{newsgroups_train.data[idx]}"',end="\n\n")

# Repetimos el proceso anterior
cossim = cosine_similarity(X_train[idx], X_train)[0] # Por similitud de coseno, verificamos similaridad con el resto de los datos
simil = np.sort(cossim)[::-1][1:6] # Almacenamos la medida de similaridad para cada documento
mostsim = np.argsort(cossim)[::-1][1:6] # Ordenamos los resultados y nos quedamos con los índices de los 5 coumentos más parecidos
print("SIMILARES:")
for doc, i in enumerate(mostsim):
    print(f"#{doc} Índice {i} | Similaridad: {simil[doc]:.4f}")
    print("Clase:", newsgroups_train.target_names[y_train[i]])
    print("Contenido:")
    print(f'"{newsgroups_train.data[i]}"',end="\n\n")

DOCUMENTO ANALIZADO: 3375
Clase: comp.sys.ibm.pc.hardware
Contenido:
"
Then don't complain (maybe it wasn't you) that SCSI was so expensive on
PC's because all we've had until a year or two ago was the ISA bus.
(ie no one buys SCSI for ISA because ISA is slow)
Are you saying that SCSI on an ISA bus is not an automatic winner when
compared to IDE?


I get different transfer rates out of my IDE when I change my ISA bus speed.


IDE is just a variant of the old IBM- MFM AT controller.  (at least that's
how it looks from a software point of view).  It was never meant to be
an all-encompassing protocal/standard to be implimented across different
platforms.

Is there any argument that 
IDE can (or can't) transfer data from the IDE drive at least as fast as the
drive is able to provide the data?  Are SCSI versions of IDE drives able
to deliver higher sustained transfer rates to their SCSI interface (because
of a higher RPM platter, different arrangement of heads, etc?)?


If data is going fro


**Análisis:**
- **Original:** 
    - _Términos clave: _
- **Similitud 0** (`a`): 
    - _Términos en común: _
- **Similitud 1** (`a`): 
    - _Términos en común: _
- **Similitud 2** (`a`): 
    - _Términos en común: _
- **Similitud 3** (`a`): 
    - _Términos en común: _
- **Similitud 4** (`a`): 
    - _Términos en común: _

---